# Try out notebook for TOME vault web scraper

## Observations
- base url of page list: https://te4.org/characters-vault
- url of page n: https://te4.org/characters-vault?page=n
- links of character pages can be extracted from page list
- talents need to be extracted
- somehow extract talents that need to be unlocked?
- stats too
- gear too?
- stats too?
- inscriptions too?
- Only selected permadeth = 'roguelike': https://te4.org/characters-vault?tag_name=&tag_level_min=&tag_level_max=&tag_permadeath%5B%5D=66#
- only selected difficulty = 'insane': https://te4.org/characters-vault?tag_name=&tag_level_min=&tag_level_max=&tag_difficulty%5B%5D=36#
- only selected race = 'cornac': https://te4.org/characters-vault?tag_name=&tag_level_min=&tag_level_max=&tag_race%5B%5D=8#
- only selected class = 'archmage': https://te4.org/characters-vault?tag_name=&tag_level_min=&tag_level_max=&tag_class%5B%5D=7#
- only selected campagin = 'majeyal': https://te4.org/characters-vault?tag_name=&tag_level_min=&tag_level_max=&tag_campaign%5B%5D=2#
- only selected version = '1.7.6': https://te4.org/characters-vault?tag_name=&tag_level_min=&tag_level_max=&tag_game%5B%5D=1191241#
- Can't immediately see maximum number of pages
- should probably build in a way to cap the number of characters extracted
- should somehow solve the issue of chinese characters




In [238]:
# Load packages

from bs4 import BeautifulSoup
import requests
import pandas as pd

### Extract all character links from a page

In [239]:
## Set up
base_url = "https://te4.org/characters-vault"

req = requests.get(base_url)
print(req)

soup = BeautifulSoup(req.text, 'html.parser')

## One character urls
char_url_html = soup.find("tr", {"class": "even"})
char_url = "https://te4.org/" + char_url_html.find_all("a")[1].get("href")
print(char_url)

## All character urls of page
char_url_html_list = soup.find_all("tr", {"class": "even"}) + soup.find_all("tr", {"class": "odd"})

char_url_list = set()

# Loop over all html
for url_html in char_url_html_list:
    char_url_list.add("https://te4.org/" + url_html.find_all("a")[1].get("href"))
    
print(char_url_list)

# Method that gets the character urls from a page
def get_char_urls_from_page(page_url=None, soup=None):
    
    # Set up BeautifulSoup if isn't given
    if not soup:
        req = requests.get(page_url)
        soup = BeautifulSoup(req.text, 'html.parser')
    
    # Extract the html elements that contain the urls
    char_url_html_list = soup.find_all("tr", {"class": "even"}) + soup.find_all("tr", {"class": "odd"})
    
    # Loop over those elements to get the character page urls
    char_url_list = set()
    for url_html in char_url_html_list:
        char_url_list.add("https://te4.org/" + url_html.find_all("a")[1].get("href"))
        
    # Return set
    return char_url_list

<Response [200]>
https://te4.org//characters/250682/tome/76d5b7cb-8c04-4485-abef-7c9fa72418e5
{'https://te4.org//characters/216966/tome/c684076c-76b8-443b-870f-9079b639446c', 'https://te4.org//characters/310510/tome/8a1719e1-76d0-4388-b0cb-38322f231fcf', 'https://te4.org//characters/289437/tome/5c26f20b-51e6-49c4-ae27-eac4c7d9be49', 'https://te4.org//characters/175357/tome/692f6cc2-0fe0-4cb0-9ff7-6efda61de1bf', 'https://te4.org//characters/246445/tome/a555aa1f-8f87-49cd-a8bf-e9923123ff4d', 'https://te4.org//characters/334280/tome/088f1bb1-15a4-4b4a-965d-348f58e14ab8', 'https://te4.org//characters/298093/tome/5927a59b-4a2b-4d20-8eae-cff6823d2cb2', 'https://te4.org//characters/310172/tome/39b6977a-0fff-4cef-a88f-b6581d859141', 'https://te4.org//characters/312327/tome/0a5ff848-6c1f-4acb-a2b9-4357d887de6b', 'https://te4.org//characters/234492/tome/1e75bba1-131e-4f36-9d90-40f4de4b5448', 'https://te4.org//characters/270836/tome/fb5e787b-b938-4e47-8fc3-1383ad75fe60', 'https://te4.org//charact

### Extract character urls from multiple pages, pages start at 0

In [240]:
base_url = "https://te4.org/characters-vault?tag_name=&tag_level_min=&tag_level_max=&tag_game%5B%5D=1191241&page=1"
empty_url = "https://te4.org/characters-vault?tag_name=&tag_level_min=&tag_level_max=&tag_winner=winner&tag_permadeath%5B%5D=66&tag_difficulty%5B%5D=36&tag_race%5B%5D=47&tag_class%5B%5D=104071&tag_game%5B%5D=1191241#"

# Skip if 'no characters available' shows up or url_limit is reached
req = requests.get(empty_url)

soup = BeautifulSoup(req.text, "html.parser")

check = soup.find("tr", {"class":"odd"})
if check.text == 'No characters available. ':
    print('no')
    
def empty_page(page_url=None, soup=None):
    if not soup:
        req = requests.get(page_url)
        soup = BeautifulSoup(req.text, 'html.parser')
        
    check = soup.find("tr", {"class":"odd"})
    if check.text == 'No characters available. ':
        return True
    else:
        return False
    
# Method that returns all the character urls, up to a maximum
def get_all_character_urls(base_url, max_urls = 100):
    print('Extracting character urls...')
    
    # Set up
    character_urls = set()
    page_number = 0
    
    while len(character_urls) <= max_urls:
        
        print(f'Extracting characters from page {page_number}...')
        
        # Make current page url, get soup
        page_url = f"{base_url}&page={page_number}"
        
        req = requests.get(page_url)
        soup = BeautifulSoup(req.text, "html.parser")
        
        # Break if the page is empty
        if empty_page(soup=soup):
            print(f"Page {page_number} is empty. Ending...")
            break
        
        # Get the character urls from the current page
        character_urls = character_urls | get_char_urls_from_page(soup=soup) # Take union of the two sets
        
        # Update the page number
        page_number += 1
        
    return character_urls
    
urls = get_all_character_urls(base_url)


KeyboardInterrupt: 

### Extracting info from character page

In [254]:
char_url = "https://te4.org/characters/259208/tome/bddd99b8-9594-44d2-a0ba-ec743bafabfa"

req = requests.get(char_url)

soup = BeautifulSoup(req.text, 'html.parser')

### Name of the character (and the creator)
full_name = soup.find("div", {"id": "title-container"}).text

### Info from tables at the top
char_tables = soup.find_all("div", {"class": "charsheet"})

def get_table_dict(tables):
    
    dict = {}
    
    dict['Prodigies'] = 17
    
    for index, table in enumerate(tables):
        try:
            name = table.find('h4').text
        
            dict[name] = index
        except:
            pass
        
    return dict

## Character table (Maybe change to dictionary later)
character_table = char_tables[0]
character_table_entries = character_table.find_all("tr")

# Game & Version
game_line = character_table_entries[0]
game_text = game_line.find_all("td")[1].text
game_text_split = game_text.split(' ')

version = list.pop(game_text_split)
game = ' '.join(game_text_split)

# Difficulty and permadeatch
mode_line = character_table_entries[1]
mode_text = mode_line.find_all("td")[1].text
mode_text_split = mode_text.split(' ')
difficulty = mode_text_split[0]
permadeath = mode_text_split[1]
print(difficulty, permadeath)
# Sex
sex_line = character_table_entries[2]
sex = sex_line.find_all("td")[1].text

# Race
race_line = character_table_entries[3]
race = race_line.find_all("td")[1].text

# Class
class_line = character_table_entries[4]
class_ = class_line.find_all("td")[1].text

# Level
level_line = character_table_entries[5]
level = level_line.find_all("td")[1].text

## Stats
stats_table = char_tables[1]
stats_table_entries = stats_table.find_all("tr")

stats = {}
for row in stats_table_entries:
    stat = row.find_all("td")[0].text
    value = row.find_all("td")[1].text
    stats[stat] = value
    
print(stats)
    
## Infusions
infusions_table = char_tables[14]
infusions_html = infusions_table.find_all("td", {"class": "qtip-link"})

infusions = list()
for infusion in infusions_html:
    infusion.find('div').decompose()
    infusions.append(infusion.text)
    
print(infusions)

## Class and Generic Talents
from collections import OrderedDict

def get_trees(table):
    
    try:
        lines_html = table.find_all("tr")
        
        talents = OrderedDict()
        line_num = 0
        extra = 0

        # Loop over all lines
        for line in lines_html:
            
            # Every fifth line stands for a tree
            if line_num % (5 + extra) == 0:
                line_num = 0
                elements = line.find_all('td')
                tree = elements[0].text

                tree_dict = OrderedDict()
                
                if tree == 'Technique / Combat training':
                    extra = 2 
                else: 
                    extra = 0
            
            # The other lines stand for skills in a tree
            else:
                line.find('div').decompose()
                talent = line.find('li').text
                level = line.find_all('td')[-1].text
                level_int = int(level[0])
                tree_dict[talent] = level_int
                talents[tree] = tree_dict

            line_num += 1
            
        return talents
    
    except Exception as e:
        print('Something went wrong extracting the skill tree')
        print(e)
        
class_talents_table = char_tables[15]
generic_talents_table = char_tables[16]

class_talents = get_trees(class_talents_table)
generic_talents = get_trees(generic_talents_table)

print(class_talents)
print(generic_talents)

# Prodigies
prodigy_table = char_tables[17]
print(prodigy_table.find("h4"))
entries = prodigy_table.find_all('tr')

prodigies = list()
for line_html in entries:
    line_html.find('div').decompose()
    prodigies.append(line_html.find('li').text)
    
print(prodigies)

# Method that puts the relevant data of a character in a dictionary
def get_character_dictionary(char_url):
    
    print(f'Beginning to extract {char_url}...')
    
    # Set up BeautifulSoup
    req = requests.get(char_url)
    soup = BeautifulSoup(req.text, 'html.parser')

    ### Name of the character (and the creator)
    full_name = soup.find("div", {"id": "title-container"}).text

    ### Info from tables at the top
    char_tables = soup.find_all("div", {"class": "charsheet"})
    
    tables_dict = get_table_dict(char_tables)

    ## Character table (Maybe change to dictionary later)
    character_table = char_tables[0]
    character_table_entries = character_table.find_all("tr")

    # Game & Version
    game_line = character_table_entries[0]
    game_text = game_line.find_all("td")[1].text
    game_text_split = game_text.split(' ')

    version = list.pop(game_text_split)
    game = ' '.join(game_text_split)

    # Difficulty and permadeatch
    mode_line = character_table_entries[3]
    mode_text = mode_line.find_all("td")[1].text
    mode_text_split = mode_text.split(' ')
    difficulty = mode_text_split[0]
    permadeath = mode_text_split[1]

    # Sex
    sex_line = character_table_entries[4]
    sex = sex_line.find_all("td")[1].text

    # Race
    race_line = character_table_entries[5]
    race = race_line.find_all("td")[1].text

    # Class
    class_line = character_table_entries[6]
    class_ = class_line.find_all("td")[1].text

    # Level
    level_line = character_table_entries[7]
    level = level_line.find_all("td")[1].text

    ## Stats
    stats_table = char_tables[1]
    stats_table_entries = stats_table.find_all("tr")

    stats = {}
    for row in stats_table_entries:
        stat = row.find_all("td")[0].text
        value = row.find_all("td")[1].text
        stats[stat] = value
        
    ## Inscriptions
    inscriptions_table = char_tables[14]
    inscriptions_html = inscriptions_table.find_all("td", {"class": "qtip-link"})

    inscriptions = list()
    for inscription in inscriptions_html:
        inscription.find('div').decompose()
        inscriptions.append(inscription.text)

    ## Class and Generic Talents
    
    class_talents_index = tables_dict["Class Talents"]
    generic_talents_index = tables_dict["Generic Talents"]
    
    class_talents_table = char_tables[class_talents_index]
    generic_talents_table = char_tables[generic_talents_index]
    
    class_talents = get_trees(class_talents_table)
    generic_talents = get_trees(generic_talents_table)

    ## Prodigies
    prodigies = list()
    
    prodigy_index = tables_dict['Prodigies']
    prodigy_table = char_tables[prodigy_index]
    
    if prodigy_table.find("h4").text == "Prodigies":
        
        entries = prodigy_table.find_all('tr')
        
        for line_html in entries:
            line_html.find('div').decompose()
            prodigies.append(line_html.find('li').text)
        
    char_dictionary = {'name': full_name,
                       'race': race,
                       'class': class_,
                       'sex': sex,
                       'level': level,
                       'stats': stats,
                       'inscriptions': inscriptions,
                       'class talents': class_talents,
                       'generic talents': generic_talents,
                       'prodigies': prodigies,
                       'game': game,
                       'version': version,
                       'difficulty': difficulty,
                       'permadeath': permadeath}
    
    return char_dictionary



Items Vault
{'Strength': '51 (base 9)', 'Dexterity': '46 (base 30)', 'Constitution': '138 (base 60)', 'Magic': '136 (base 62)', 'Willpower': '23 (base 9)', 'Cunning': '82 (base 60)'}
['Infusion: Movement', 'Rune: Mirror Image', 'Infusion: Wild Growth', 'Rune: Dissipation', 'Rune: Shatter Afflictions']
OrderedDict([('Celestial / Circles', OrderedDict([('Circle of Shifting Shadows', 5), ('Circle of Sanctity', 5), ('Circle of Warding', 3), ('Celestial Surge', 4)])), ('Celestial / Star fury', OrderedDict([('Moonlight Ray', 5), ('Shadow Blast', 5), ('Twilight Surge', 5), ('Starfall', 1)])), ('Celestial / Eclipse', OrderedDict([('Blood Red Moon', 5), ('Totality', 4), ('Corona', 1), ('Darkest Light', 5)])), ('Celestial / Sunlight', OrderedDict([('Searing Light', 5), ('Sun Flare', 5), ('Firebeam', 0), ('Sunburst', 0)])), ('Celestial / Twilight', OrderedDict([('Twilight', 1), ('Jumpgate', 5), ('Mind Blast', 1), ('Shadow Simulacrum', 5)]))])
OrderedDict([('Race / Dwarf', OrderedDict([('Resilienc

### Filter using search link

#### Automatically get the race codes

In [ ]:
race_codes = {}

base_url = "https://te4.org/characters-vault?tag_name=&tag_level_min=&tag_level_max="

for i in range(1, 100): 
    tag = f'&tag_race%5B%5D={i}#'
    url = base_url + tag
    
    char_link = get_all_character_urls(url, max_urls=1)
    
    if char_link:
        
        try:
            char = get_character_dictionary(list(char_link)[0])
            race_codes[char['race']] = str(i)
        except: 
            print('something went wrong with this character, probably a very janky character') 

#### Automatically get class codes

In [257]:
base_url = "https://te4.org/characters-vault?tag_name=&tag_level_min=&tag_level_max="

class_codes = {}

for i in range(1, 200): 
    tag = f'&tag_class%5B%5D={i}#'
    url = base_url + tag
    
    char_link = get_all_character_urls(url, max_urls=1)
    
    if char_link:
        
        try:
            char = get_character_dictionary(list(char_link)[0])
            class_codes[char['class']] = str(i)
        except: 
            print('something went wrong with this character, probably a very janky character') 

Extracting character urls...
Extracting characters from page 0...
Page 0 is empty. Ending...
Extracting character urls...
Extracting characters from page 0...
Page 0 is empty. Ending...
Extracting character urls...
Extracting characters from page 0...
Page 0 is empty. Ending...
Extracting character urls...
Extracting characters from page 0...
Beginning to extract https://te4.org//characters/33258/tome/41a0f4bc-7165-4f0e-8f78-7a2a284e5b02...
{'Prodigies': 17, 'Character': 0, 'Primary Stats': 1, 'Resources': 2, 'Speed': 3, 'Vision': 4, 'Offense: Mainhand': 5, 'Offense: Spell': 6, 'Offense: Mind': 7, 'Defense: Base': 8, 'Defense: Immunities': 9, 'Inscriptions (3/3)': 10, 'Class Talents': 11, 'Generic Talents': 12, 'Effects': 13, 'Quests': 14, 'Equipment': 15, 'Inventory': 16, 'Achievements': 17, 'Log': 18}
Extracting character urls...
Extracting characters from page 0...
Page 0 is empty. Ending...
Extracting character urls...
Extracting characters from page 0...
Page 0 is empty. Ending...

KeyboardInterrupt: 

#### Filtering

In [256]:
import filter_codes

# Tag order: name, minlevel, maxlevel, dead, winner, permadeath, difficulty, race, class, campagin, version

https://te4.org/characters-vault?tag_name=&tag_level_min=&tag_level_max=&tag_race%5B%5D=1#
Extracting character urls...
Extracting characters from page 0...
Page 0 is empty. Ending...
set()
https://te4.org/characters-vault?tag_name=&tag_level_min=&tag_level_max=&tag_race%5B%5D=2#
Extracting character urls...
Extracting characters from page 0...
Page 0 is empty. Ending...
set()
https://te4.org/characters-vault?tag_name=&tag_level_min=&tag_level_max=&tag_race%5B%5D=3#
Extracting character urls...
Extracting characters from page 0...
{'https://te4.org//characters/287997/tome/cfad34a3-6de0-493b-917b-466c9e76d7d4', 'https://te4.org//characters/275279/tome/a6b6823f-2055-41f0-bd4e-3588ae2a5046', 'https://te4.org//characters/323064/tome/585d0496-d058-4efc-904f-7701c45a3f22', 'https://te4.org//characters/311147/tome/44afb9dd-9c8b-418c-beaa-89b67966f0c8', 'https://te4.org//characters/256622/tome/41eec381-c180-4fd4-b31c-6422903309f8', 'https://te4.org//characters/256622/tome/0a06e777-c71e-4c06-86

In [258]:
class_codes

{'Wyrmic': '4',
 'Archmage': '7',
 'Cursed': '10',
 'Rogue': '12',
 'Archer': '14',
 'Berserker': '16',
 'Summoner': '17',
 'Alchemist': '19',
 'Anorithil': '20',
 'Arcane Blade': '22',
 'Shadowblade': '23',
 'Sun Paladin': '27',
 'Doomed': '29',
 'Geomancer': '30',
 'Reaver': '31',
 'Pyromancer': '32',
 'Corruptor': '34',
 'Tutorial Adventurer': '39',
 'Paradox Mage': '43',
 'Mindslayer': '48',
 'Temporal Warden': '49',
 'Golem': '51',
 'Brawler': '56',
 'Necromancer': '68',
 'Stone Warden': '70',
 'Marauder': '71',
 'Bulwark': '80',
 'Tourist': '83',
 'Ranger': '91',
 'Illusionist': '92',
 'Monk': '97',
 'Magebow': '98',
 'Barbarian': '99',
 'Solipsist': '102',
 'Soul Stealer': '103',
 'Adventurer': '104',
 'Gardener': '107',
 'Charlatan': '108',
 'Phoenix Knight': '112',
 'Elementist': '124',
 'Werebeast': '126',
 'Rodent': '129',
 'Human': '131'}